<a href="https://colab.research.google.com/github/habilg/text-basics/blob/master/02A_word_representation(embedding)_high_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word representation(embedding)

This a way to describe every word with features to find their anology. it is not a matter this describing features are meaningfull for human. 
There are some methods to achieve it. But generally two main aproachs are:


> static embeding  
> dynamic embeding

maybe you think if we collect all the world words in a very huge corpus, we could find the unique final vector for every vocabulary.In fact, it is not the case, since words relationship are specific for every context, so we expect that same word embeding vector become diffrent from one context to another context.

static embedings are representation of words which obtained from a very large corpus. like fasttext, GloVe. However, dynamic representation is obtained form the text.

## What is Keras soloution?
it provides an `embedding layer`, which is trainable. It means that this layer could initilized with static weights or it could find proper weights during the trainig. 
input for this layer must be a type of `tf.keras.layers.TextVectorization, tf.keras.layers.StringLookup, or tf.keras.layers.IntegerLookup`.

In [7]:
from tensorflow.keras.layers import Embedding,TextVectorization
import numpy as np

## Using pretrained weights-GloVe:

embedding_initializer argument must be initialized by a embedding matrix of the training words

In [2]:
dataset = [
  "I write, erase, rewrite",
  "Erase again, and then",
  "A poppy blooms.",
]

In [3]:


text_vectorization = TextVectorization(output_mode="int")
text_vectorization.adapt(dataset)

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

In [4]:
max_tokens=100
word_index

{'': 0,
 '[UNK]': 1,
 'erase': 2,
 'write': 3,
 'then': 4,
 'rewrite': 5,
 'poppy': 6,
 'i': 7,
 'blooms': 8,
 'and': 9,
 'again': 10,
 'a': 11}

In [12]:
def embedding_for_vocab(filepath, word_index,embedding_dim):
    vocab_size = len(word_index) + 1
      
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, vector = line.split(maxsplit=1)
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.fromstring(vector, "f", sep=" ")
  
    return embedding_matrix_vocab

In [14]:
# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix= embedding_for_vocab('/content/drive/MyDrive/glove.6B.50d.txt',
                                             word_index,
                                             embedding_dim)
  
print("Dense vector for first word is => ", embedding_matrix[2])

Dense vector for first word is =>  [-0.14342999 -0.21572     0.95021999 -0.060348   -0.19272     0.59243
  0.22474     0.34395999 -0.19253001  0.41418999 -0.60180998  0.15369999
 -0.41172999 -0.46133     0.34940001  0.56075001 -0.16425    -1.12510002
  0.95339     0.058167   -0.47325    -0.55941999  0.018379   -0.81616998
  0.28639001 -0.80637002 -0.91731     0.012369    1.03820002 -0.80427003
  1.00250006  0.59998    -0.30706999 -0.27456999 -0.54820001  0.48576
 -0.44499999 -0.16799     0.67354    -1.19289994 -0.29357001 -0.69257998
  0.025876    0.27287999 -0.26589     0.78315002  0.5589      0.60016
  0.16568001 -0.90461999]


In [11]:
from tensorflow.keras.initializers import Constant

embedding_layer = Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

### Make trainable a pretrained initializer

## Training the Layer from the ground.

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

voc_size=100
onehot_repr=[one_hot(words,voc_size)for words in dataset]
onehot_repr

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sent_length=8
encoded_sentences=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
encoded_sentences

In [ ]:
from tensorflow.keras.models import Sequential


embedding_dim=10
model=Sequential()
model.add(Embedding(voc_size,embedding_dim,input_length=sent_length,mask_zero=True))
model.compile('adam','mse')

In [ ]:
model.summary()

In [ ]:
print(encoded_sentences[0])
print(model.predict(encoded_sentences[0]))

In [ ]:
print(encoded_sentences[0])
print(model.predict(encoded_sentences[0]))